In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import ExcelWriter #este va servir para exportar achivos a excel
import xlrd

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_digits
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

In [9]:
#para encontrar una grafica de excel y ponerla en python 
datos = "ML-Wells.xlsx" #busco el archivo y lo pego con el indicativo por ejemplo .xlsx
df = pd.read_excel(datos, sheet_name="T6")#sacar del excel a python
df.drop(columns= ["PHOTO","WELL","Unnamed: 0"], inplace=True) #para borrar una columna que no queremos
df.sort_values(by="DEPTH")
#print(df.shape) #para saber el tamaño de como quedo antes de borrar los valore nulos
df.dropna(inplace=True)
df.set_index(np.array(range(0,(df.shape)[0])), inplace=True)
print(df.shape) #para saber el tamaño de como quedo despues de borrar 
#df.info()
df.head()

(216, 7)


,DEPTH,GR_EDTC,RHOZ,AT90,NPHI,DTCO,GRAY
0,3484.5061,88.9094,2.4670,8.8005,0.3477,92.01340,0.0
1,3484.0056,84.1737,2.4635,8.8252,0.3383,91.57937,0.0
2,3483.5027,85.9207,2.4642,9.2523,0.3422,91.48260,0.0
3,3483.0022,85.2922,2.4670,9.7221,0.3252,91.13451,0.0
4,3482.5049,89.9095,2.4710,9.6451,0.3186,91.07773,0.0


In [12]:
gray = np.array(df.loc[:,'GRAY']) # crea un vector nuev0 todas las filas que tenga la columna GRAY
cutoff = np.zeros(gray.shape) 
# hacemos el condicional
cut_off = 250
for i in range (gray.shape[0]):
  if (gray[i] >= 140):
    cutoff[i] = 1

df['cutoff'] = cutoff
df["cutoff"].value_counts() #esto sirve para saber cuantos valores de 1 y de 0 que hay en el cutoff hay 
#df.head()

0.0    190
1.0     26
Name: cutoff, dtype: int64

In [13]:
#Definición de Vector de Salida y Partición en Train y Test Stratified
from sklearn.model_selection import StratifiedShuffleSplit #la dicision de 70 y 30 halla quede propocional los valores de salida de 100% proporsional en ambos parametros de porcentaje
from sklearn.preprocessing import StandardScaler #esto sirve para normalizar los datos osea sacar el max y el minimo yy dividir todos los valores por el maximo para todas las columnas queden en un ranco de 0 a 1

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)#tamaño de prueba
for train_index, test_index in split.split(df, df["cutoff"].values):
    Xtrain_strat = df.loc[train_index]
    Xtest_strat = df.loc[test_index]

ytrain_strat = np.array(Xtrain_strat['cutoff'])
ytest_strat = np.array(Xtest_strat['cutoff'])
Xtrain_strat.drop(columns='cutoff', inplace=True)
Xtest_strat.drop(columns='cutoff', inplace=True)

#Normalización de los datos
normalizar = StandardScaler()
Xtest_strat = normalizar.fit_transform(Xtest_strat)

print("Tamaño de Entrenamiento: ",Xtrain_strat.shape[0])
print("Tamaño de Prueba: ",Xtest_strat.shape[0])

Tamaño de Entrenamiento:  151
Tamaño de Prueba:  65


In [14]:
#Definición de Pasos y Parámetros
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV,cross_val_score, cross_val_predict 
from sklearn.metrics import mean_absolute_error, confusion_matrix, roc_auc_score
from sklearn.pipeline import Pipeline
import numpy as np
import joblib
# pasos que debe hacer con los datos #pca es un algoritmo para hacer reduccion de dimecion 
steps=[[('norm', StandardScaler()),
        ('rep', PCA()),
       ('class', RandomForestClassifier())],
       [('norm', StandardScaler()),
        ('class', RandomForestClassifier())],
       [('norm', StandardScaler()),
        ('rep', PCA()),
        ('class', SVC())],
       [('norm', StandardScaler()),
        ('class', SVC())],]
# con que parametros quiero que haga lo que esta arriba
parameters = [{'rep__n_components':[2,4,7],
              'class__max_depth': [8,10,12,15],
              'class__n_estimators' : [20,40,60,80,100]
              }, 
              {'class__max_depth': [8,10,12,15],
              'class__n_estimators' : [20,40,60,80,100]
              },
              {'rep__n_components':[2,4,6,7],
              'class__C': [1,20,50,100,150],
              'class__gamma': [0.01, 0.001, 0.0001]
              },
              {'class__C': [1,20,50,100,150],
               'class__gamma': [0.01, 0.001, 0.0001]
              }]

cross_scores_strat = {} #puntaje de precision
err_strat = []
conf_matrxs_strat = []
best_model_strat = []
filename_strat = 'modelos_class'#'resultados/people_comp'
tests_strat = ['PCA - RandomForest','Random Forest','PCA - SVC','SVC']

for i in range (len(steps)):
    pipeline = Pipeline(steps = steps[i])

    grid_search = GridSearchCV(pipeline, parameters[i], n_jobs=-1,cv=3,
                               scoring='balanced_accuracy',verbose=5) #balanceado
    
    print("\nEntrenando: {} \n".format(tests_strat[i]))
    grid_search.fit(Xtrain_strat, ytrain_strat)
    
    #mejor modelo entrenado
    best_model_n = grid_search.best_estimator_
    best_model_strat += [grid_search.best_estimator_]
    joblib.dump(best_model_strat,filename_strat+".pkl")
    print("\nMejores parámetros con {} : {}".format(tests_strat[i],grid_search.best_params_))
    
    #evaluación:
    ytest_pred = cross_val_predict(best_model_n, Xtest_strat, ytest_strat, cv=3)
    cross_score = np.mean(cross_val_score(best_model_n, Xtest_strat, ytest_strat, cv=3, scoring="balanced_accuracy"))
    cross_scores_strat[tests_strat[i]] = cross_score
    conf_matrx = confusion_matrix(ytest_strat, ytest_pred, normalize= 'true')

    print("\nCros-Score con {} : {:.3f}\n".format(tests_strat[i],cross_score))
    err_strat.append(cross_score)
    conf_matrxs_strat.append(conf_matrx)


Entrenando: PCA - RandomForest 

Fitting 3 folds for each of 60 candidates, totalling 180 fits

Mejores parámetros con PCA - RandomForest : {'class__max_depth': 12, 'class__n_estimators': 20, 'rep__n_components': 7}

Cros-Score con PCA - RandomForest : 0.694


Entrenando: Random Forest 

Fitting 3 folds for each of 20 candidates, totalling 60 fits

Mejores parámetros con Random Forest : {'class__max_depth': 8, 'class__n_estimators': 20}

Cros-Score con Random Forest : 0.889


Entrenando: PCA - SVC 

Fitting 3 folds for each of 60 candidates, totalling 180 fits

Mejores parámetros con PCA - SVC : {'class__C': 100, 'class__gamma': 0.001, 'rep__n_components': 6}

Cros-Score con PCA - SVC : 0.871


Entrenando: SVC 

Fitting 3 folds for each of 15 candidates, totalling 45 fits

Mejores parámetros con SVC : {'class__C': 100, 'class__gamma': 0.001}

Cros-Score con SVC : 0.871



In [15]:
print("Validación: \n", cross_scores_strat)

Validación: 
 {'PCA - RandomForest': 0.6944444444444443, 'Random Forest': 0.8888888888888888, 'PCA - SVC': 0.871345029239766, 'SVC': 0.871345029239766}
